# TP1 - Grupo 14

André Lucena Ribas Ferreira - A94956

Paulo André Alegre Pinto - A97391

## Problema 2 - Cálculo do SVP

Pretende-se alocar reuniões de Projetos a certas Salas, dentro de determinados 'Timeslots', aos Colaboradores de uma StartUp.

Como 'inputs', o problema recebe:
1. O número de 'Timeslots' $T$;
2. O número de Salas $S$;
3. O número de Projetos $P$;
4. O número de Colaboradores $C$;
5. O conjunto de Colaboradores de cada Projeto, o seu Líder e o número mínimo de reuniões semanais, num dicionário $proj$.
6. A disponibilidade de cada Colaborador. Essa disponibilidade é representada numa matriz booleana de acessibilidade $disp$.

### Análise

Para resolver o problema do SVP, decidimos usar duas famílias de variáveis, uma binária e outra inteira.

A família de variáveis binárias $e_{}$ é definida com a seguinte semântica:

colocar isto bonito:
e[j][0] se e só se e[j] == -1
e[j][1] se e só se e[j] == 0
e[j][2] se e só se e[j] == 1

A família de variáveis inteiras $k_{}$ servirá para conseguir iterar pelos números inteiros, de modo a se encontrar os múltiplos de $q$ para cada resultado da multiplicação.

In [ ]:
from ortools.linear_solver import pywraplp
import numpy

#  n  > 30
# |m| > 1 + |n|
# |q| > |m|

n = 5
m = 16
q = 37

def SVP(m,n,q):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    L = numpy.random.randint(-(q-1)/2, (q-1)/2, (m, n))
    e = {}
    k = {}
    #e é um vetor de tamanho m com valores inteiros -1, 0 ou 1
    for j in range(m):
        e[j] = [solver.BoolVar('e[%i][-1]' % j),
                solver.BoolVar('e[%i][0]' % j),
                solver.BoolVar('e[%i][1]' % j)]
        #e só pode ter um elemento por cada j
        solver.Add(e[j][0] + e[j][1] + e[j][2] == 1)
    
            
    #k é um vetor de tamanho n com valores inteiros
    for i in range(n):
        k[i] = solver.IntVar(-solver.infinity(), solver.infinity(), 'k[%i]' % i)
    
    #1. Tem de existir um elemento não nulo em e
    solver.Add(sum([e[j][1] for j in range(m)]) <= m-1)
    
    #2. Multiplicar e por L deve resultar num vetor de múltiplos de q
    for i in range(n):
        solver.Add((sum([(e[j][2] - e[j][0])*L[j][i] for j in range(m)])) == q * k[i])
    
    solver.Maximize(sum([e[j][1] for j in range(m)]))
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        print(L)
        print("")
        for i in range(n):
            print(k[i].solution_value(), end = " ")
        print("")
        for j in range(m):
            for k in range(3):
                if e[j][k].solution_value():
                    print(k-1, end = " ")
        return True
    else:
        print(L)
        print("Não é possível resolver o problema")
        return False
    
count = 0
while (not SVP(m,n,q) and count < 10):
    count += 1
print(f'Falhou {count} vezes.')

###### 